In [1]:
from models import FourLayerModel
from data import DataSource

In [2]:
X_train, Y_train, X_test, Y_test, desc = DataSource().get_training_data()

In [4]:
TRAIN_EPOCHS = 15001
LEARNING_RATE = 0.001
PRINT_EACH = 1000

In [5]:
nn = FourLayerModel(X_train.shape[0], 5, 4)

In [6]:
for i in range(0, TRAIN_EPOCHS):
    predictions = nn.forward_propagation(X_train)
    cost = nn.calculate_cost(predictions, Y_train)

    nn.backward_propagation(X_train, Y_train)

    nn.update_weights(LEARNING_RATE)

    if (i) % PRINT_EACH == 0:
        print("Cost after iteration %i: %f " % (i, cost))

Cost after iteration 0: 601.210960 
Cost after iteration 1000: 7.917978 
Cost after iteration 2000: 7.495592 
Cost after iteration 3000: 6.847989 
Cost after iteration 4000: 6.512429 
Cost after iteration 5000: 6.316242 
Cost after iteration 6000: 6.199060 
Cost after iteration 7000: 6.104805 
Cost after iteration 8000: 6.063787 
Cost after iteration 9000: 6.036635 
Cost after iteration 10000: 6.019376 
Cost after iteration 11000: 6.007958 
Cost after iteration 12000: 5.997156 
Cost after iteration 13000: 5.990236 
Cost after iteration 14000: 5.984817 
Cost after iteration 15000: 5.979764 


In [7]:
test_predictions = nn.forward_propagation(X_test)

print("Predictions: ", test_predictions.astype(int))
print("Real Values: ", Y_test.astype(int))

Predictions:  [[29 26 30 32 25 28 42 34 29 28]]
Real Values:  [[26 22 32 36 27 27 44 32 28 31]]
